In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import print_function
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

import scipy
from scipy import stats
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
from functools import partial
#from tqdm import tqdm
from tqdm.autonotebook import tqdm

from ipywidgets import interact, interactive, fixed, interact_manual


# DATA VISUALIZATION AND EXPLORATION

In [ ]:
dataset1_path = "datasets/1.csv"
dataset1 = pd.read_csv(dataset1_path)
dataset2_path = "datasets/2.csv"
dataset2 = pd.read_csv(dataset2_path)
dataset3_path = "datasets/3.csv"
dataset3 = pd.read_csv(dataset3_path)
dataset4_path = "datasets/4.csv"
dataset4 = pd.read_csv(dataset4_path)

In [ ]:
dataset12 = pd.concat([dataset1, dataset2.add([2, 0, 0, 0])])

In [ ]:
# Load dataset
dataset_path = "datasets/3.csv"
dataset = pd.read_csv(dataset_path)

In [ ]:
dataset.iloc[:5]

In [ ]:
plt.rcParams['figure.figsize'] = [5, 5]
dataset["in1"].hist(bins=40)
hist, bins = np.histogram(dataset["in1"], bins=2)
#print(hist, bins)
norm_hist = hist / len(dataset)
#scipy.stats.entropy(norm_hist)

In [ ]:
plt.rcParams['figure.figsize'] = [5, 5]
dataset["in2"].hist(bins=40)
hist, bins = np.histogram(dataset["in2"], bins=250)
#print(hist, bins)
norm_hist = hist / len(dataset)
scipy.stats.entropy(norm_hist)

In [ ]:
plt.rcParams['figure.figsize'] = [5, 5]
dataset["out1"].hist(bins=40)
hist, bins = np.histogram(dataset["out1"], bins=250)
#print(hist, bins)
norm_hist = hist / len(dataset)
scipy.stats.entropy(norm_hist)

In [ ]:
plt.rcParams['figure.figsize'] = [5, 5]
dataset["out2"].hist(bins=40)
hist, bins = np.histogram(dataset["out2"], bins=250)
#print(hist, bins)
norm_hist = hist / len(dataset)
scipy.stats.entropy(norm_hist)

In [ ]:

import sklearn
from sklearn.cluster import MiniBatchKMeans
from itertools import cycle, islice


kMeans = MiniBatchKMeans(n_clusters=16)  

dataset_path = "datasets/3.csv"
dataset = pd.read_csv(dataset_path)
X = dataset[['in2', 'out1', 'out2']].values
kMeans.fit_predict(X)
y_pred = kMeans.labels_
colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
                                             '#f781bf', '#a65628', '#984ea3',
                                             '#999999', '#e41a1c', '#dede00']),
                                      int(max(y_pred) + 1))))

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X[:, 0], X[:, 1], X[:, 2], color=colors[y_pred])
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [30, 20]
for i in range(len(dataset)):
    color = "r" if dataset.iloc[i]["in1"] == 0 else "b"
    if color != 'r':
        continue
    plt.plot(np.array(range(0, 4)), dataset.iloc[i], f"{color}-", scaley=True)
plt.figure(figsize=(10, 10))
plt.show()

In [ ]:

def color(sample, bit = 0):
    if sample & (1 << bit):
        return 'r'
    return 'b'

In [ ]:
plt.rcParams['figure.figsize'] = [20, 10]

zero_data = dataset["in1"] == 0
ones_data = dataset["in1"] == 1

for i in range(len(dataset[zero_data]["in2"])):
    plt.scatter(dataset[zero_data]["in2"].iloc[i], dataset[zero_data]["out1"].iloc[i], c=color(dataset[zero_data]["out1"].iloc[i], 1))

plt.figure(figsize=(10, 10))
plt.show()

In [ ]:
dataset.corr()

# EXPERIMENTS

The first approach was to train a neural netowkr to predict the numbers from the inputs directly but id did not work.
The next approaches switched into interperting the inputs and outpus as vectors of bits/nibbles. The following code contains
logic for octal representation of the numbers and binary representation of the values.

In [ ]:
#model = SmallestNN(2, 2) does not work
#model = SmallestNNDecoderLike(2, 2) does not not work
#model = SmallestNNDecoderLikeV2(2, 2) does not work
#model = SmallestNNDecoderLikeV2(1, 1) does not work
#model = SmallestNNBitVectV2(16, 512, 512, 16) # Epochs: 250;  Accuracy: 0.97; Time: ~5minutes
#model = SmallestCNNBitVect(16, 16) # does not learn)

In [ ]:
from dataset import *
from models import *
from experiments import *

In [ ]:
ds = SmallestNNDatasetBitEncodedToKthBit(SmallestNNDatasetBitEncodedMinusOne2One(dataset), 0)

def viz(idx):
    print("Index:", idx, ds[idx])

interact(viz, idx=(0, len(ds) - 1))

In [ ]:
dss = SmallestNNDatasetBitEncodedMinusOne2One(dataset, 2)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
dssss = SmallestNNDatasetBitEncodeNormalized2BitOneHot(dataset)

In [ ]:

class OctBaseReprExperiment(Experiment):
    def __init__(self):
        super(OctBaseReprExperiment, self).__init__(
            SmallestNNBitVectV2(4, 512, 512, 4).to(device),
            DataLoader(SmallestNNDatasetBitEncodedBaseK(dataset), batch_size=1, shuffle=True),
            DataLoader(SmallestNNDatasetBitEncodedBaseK(dataset), batch_size=1, shuffle=True),
            torch.optim.Adam,
            torch.nn.MSELoss(),
            device
        )
    
    def evaluate(self):
        #import pdb; pdb.set_trace()
        correct = 0 
        for batch in self.test_data:
            input, label = batch
            input = input.to(self.device)
            label = label.to(self.device)
            predct = self.model(input)
            
            predct = decode(dec2hex(predct.detach().cpu()).view(1, 16))
            label = decode(dec2hex(label.detach().cpu()).view(1, 16))
            correct += (predct == label).sum() / 2.0
        correct = correct.float()
        return (correct / len(self.test_data))

In [ ]:
#trn = OctBaseReprExperiment()
#trn.train()

In [ ]:
class OctBaseReprCNNExperiment(Experiment):
    def __init__(self):
        super(OctBaseReprCNNExperiment, self).__init__(
            SmallestCNNBitVect(4, 4).to(device),
            DataLoader(SmallestNNDatasetBitEncodedBaseK(dataset, 1), batch_size=1, shuffle=True),
            DataLoader(SmallestNNDatasetBitEncodedBaseK(dataset, 1), batch_size=1, shuffle=True),
            torch.optim.Adam,
            torch.nn.MSELoss(),
            device
        )
    
    def evaluate(self):
        #import pdb; pdb.set_trace()
        correct = 0 
        for batch in self.test_data:
            input, label = batch
            input = input.to(self.device)
            label = label.to(self.device)
            predct = self.model(input)
            predct = decode(dec2hex(predct.detach().cpu()).view(1, 16))
            label = decode(dec2hex(label.detach().cpu()).view(1, 16))
            correct += (predct == label).sum() / 2.0
        correct = correct.float()
        return (correct / len(self.test_data))

In [ ]:
#torch.cuda.current_device()

In [ ]:
#trn = OctBaseReprCNNExperiment()
#trn.train()

In [ ]:
class BinaryReprExperiment(Experiment):
    def __init__(
        self,
        model = SmallestNNBitVectV2(16, 352, 384, 16).to(device),
        train_data = DataLoader(SmallestNNDatasetBitEncodedMinusOne2One(dataset), batch_size=1, shuffle=True),
        test_data = DataLoader(SmallestNNDatasetBitEncodedMinusOne2One(dataset), batch_size=1, shuffle=True),
        optimizer_class = torch.optim.Adam,
        criterion = torch.nn.MSELoss(),
        device = device,
        epochs = 10000,
        decode = None
    ):
        super(BinaryReprExperiment, self).__init__(
            model, train_data, test_data, optimizer_class, criterion, device, epochs, decode
        )
    
    def evaluate(self):
        T = torch.Tensor([0.5]).to(device)  # threshold
        correct = 0
        for batch in self.test_data:
            input, label = batch # human_understandable_*
            input = input.to(device).unsqueeze(0)
            label = label.to(device)
            predct = self.model(input)[0]
            predct = (predct > T).float() * 1
            
            if self.decode:
                predct = sel.decode(predct)
            else:
                predct = decode(predct)

            if self.decode:
                label = sel.decode(label)
            else:
                label = decode(label)

            correct += (predct == label).float().sum() / 2.0

        return (correct.float() / len(self.test_data))

In [ ]:
trn = BinaryReprExperiment()
#trn.train()

In [ ]:
#torch.save(trn.model, "dataset2_ln16x320x320x16_acc1.0.pth")

In [ ]:
ds = SmallestNNDatasetBitEncodedMinusOne2One(dataset)

In [ ]:
def check_model(idx):
    print(f"DF[{idx}]=\n", dataset.iloc[idx])
    sample = ds[idx]
    print(f"Human undersntadable \nInput={decode(np.expand_dims(ds[idx][0], 0))} \nOutput={decode(np.expand_dims(ds[idx][1], 0))}")
    pred = traced_script_module(sample[0])
    print(f"raw pred {pred}")
    pred = (pred > T).float()
    print(f"refined pred {pred}")
    print(f"Human undersntadable pred:\n", decode(pred.cpu()).numpy()[0])
    print(f" {decode(ds[idx][1])[0].numpy()}")
    
    


In [ ]:
#interact(check_model, idx=(0,len(dataset)))

In [ ]:
class BinaryReprConvNNExperiment(Experiment):
    def __init__(
        self,
        model = SmallestCNNBitVect(16, 16).to(device),
        train_data = DataLoader(SmallestNNDatasetBitEncodedMinusOne2One(dataset), batch_size=1, shuffle=True),
        test_data = DataLoader(SmallestNNDatasetBitEncodedMinusOne2One(dataset), batch_size=1, shuffle=True),
        optimizer_class = torch.optim.Adam,
        criterion = torch.nn.MSELoss(),
        device = device,
        epochs = 10000,
        decode = None
    ):
        super(BinaryReprConvNNExperiment, self).__init__(
            model, train_data, test_data, optimizer_class, criterion, device, epochs
        )
    
    def train(self):
        #import pdb; pdb.set_trace()
        losses = []
        moment = []
        acc = 0

        for e in tqdm(range(self.epochs)):
            cum_loss = 0
            for batch in self.train_data:
                input, label = batch
                input = input.to(self.device)
                label = label.to(self.device).float() #.unsqueeze(1).float()
                predct = self.model(input)
                loss = self.criterion(predct, label)
                self.opt.zero_grad()
                loss.backward()
                self.opt.step()
                cum_loss += loss.item()

            curr = self.evaluate()
            if acc < curr:
                acc = curr
                print("Epoch: %5d:" % (e), curr)
            if acc >= 0.99:
                break
            self.sched.step(curr.item())

            moment.append(e)
            losses.append(cum_loss / len(self.train_data))
        self.display_learning(moment, losses)
    
    def evaluate(self):
        #import pdb; pdb.set_trace()
        T = torch.Tensor([0.5]).to(device)  # threshold
        correct = 0 
        for batch in self.test_data:
            input, label = batch # human_understandable_*
            input = input.to(device)#.unsqueeze(1)
            label = label.to(device).float() #.unsqueeze(1).float()
            predct = self.model(input)[0]
            predct = (predct > T).float() * 1
            
            if self.decode:
                predct = self.decode(predct)
            else:
                predct = decode(predct)
            
            if self.decode:
                label = self.decode(label)
            else:
                label = decode(label)
            correct += predct == label

        return (correct.float() / len(self.test_data)).sum() / 2.0

In [ ]:
#trn = BinaryReprConvNNExperiment()
#trn.train()

In [ ]:
#print(sample)
#p = trn.model(sample[0].cuda())

In [ ]:
class SmallestNNCNNBitPredictor(torch.nn.Module):
    def __init__(self):
        super(SmallestNNCNNBitPredictor, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=256, kernel_size=16)
        mdls.append(nn.Linear(256, 1))
        mdls.append(nn.ReLU())
        self.seq = nn.Sequential(*mdls)

    def forward(self, x):
        x = self.seq(x)
        return x

In [ ]:
class BinaryReprToBitExperiment(Experiment):
    def __init__(
        self,
        model = SmallestNNBitVectV2(16, 256, 2).to(device),
        train_data = DataLoader(
            SmallestNNDatasetBitEncodedToKthBit(
                SmallestNNDatasetBitEncodedMinusOne2One(dataset),
                0),
            batch_size=1,
            shuffle=True),
        test_data = DataLoader(
            SmallestNNDatasetBitEncodedToKthBit(
                SmallestNNDatasetBitEncodedMinusOne2One(dataset),
                0),
            batch_size=1,
            shuffle=True),
        optimizer_class = torch.optim.Adam,
        criterion = torch.nn.MSELoss(),
        device = device,
        epochs=1000,
        init_weigths = True,
        trainable_parameters = None
    ):
        super().__init__(
            model, train_data, test_data, optimizer_class, criterion, device,
            epochs=epochs, init_weigths=init_weigths
        )
        if not trainable_parameters:
            return
        
        self.opt = optimizer_class(trainable_parameters.parameters(), lr=1e-4)
        self.sched = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.opt, mode='min', factor=0.95, patience=10,
            verbose=False, threshold=0.01, min_lr=1e-7, eps=1e-08
        )
    
    def train(self):
        #import pdb; pdb.set_trace()
        losses = []
        moment = []
        acc = 0

        for e in tqdm(range(self.epochs)):
            cum_loss = 0
            for batch in self.train_data:
                input, label = batch
                input = input.to(self.device).unsqueeze(0).float()
                label = label.to(self.device).unsqueeze(0).float()
                predct = self.model(input)
                loss = self.criterion(predct, label)
                self.opt.zero_grad()
                loss.backward()
                self.opt.step()
                cum_loss += loss.item()

            curr = self.evaluate()
            if acc < curr:
                acc = curr
                print("\rEpoch: %5d:" % (e), curr, end='')
            if acc == 1.0:
                break
            self.sched.step(curr.item())

            moment.append(e)
            losses.append(cum_loss / len(self.train_data))
        
        #import pdb; pdb.set_trace()
        self.display_learning(moment, losses)


    def evaluate(self):
        T = torch.Tensor([0.5]).to(device)  # threshold
        correct = 0 
        for batch in self.test_data:
            input, label = batch # human_understandable_*
            input = input.to(device)
            label = label.to(device).float()
            predct = self.model(input)[0]
            predct = predct.argmax().item()
            label = label.argmax().item()
            #print(predct, label)
            correct += float(predct == label)

        return torch.Tensor([(correct / len(self.test_data))])

In [ ]:
#trn = BinaryReprToBitExperiment()
#trn.train()

In [ ]:
from pprint import pprint
train_data_explicit = SmallestNNDatasetBitEncodedMinusOne2One(dataset)
train_data = SmallestNNDatasetBitEncodedToKthBit(SmallestNNDatasetBitEncodedMinusOne2One(dataset), 0)
#print(len(train_data))

def viz(idx):
    print("Index:", idx)
    pprint(train_data_explicit[idx][0])
    pprint(train_data_explicit[idx][1])
    print("=" * 50)
    pprint(train_data[idx][0])
    pprint(train_data[idx][1])
    print("target class:", train_data[idx][1].argmax())
    out = trn.model(train_data[idx][0].unsqueeze(0).cuda())[0].cpu()
    print("predct_class:", out.argmax().item())
    print(out)

interact(viz, idx=(0, len(train_data) - 1))

### Since predicting all the 16 bits is ineficient since the algebra inside the NN multiplies matrixes o shape (input_size, output_size) we to have 16 prediction heads for each bit separately.

In [ ]:
class SmallestCNNBits(torch.nn.Module):
    def __init__(self, *sizes):
        super().__init__()
        
        mdls = []
        feature_sizes = sizes[:-1]

        for (inp, out) in zip(feature_sizes[:-1], feature_sizes[1:]):
            mdls.append(nn.Linear(inp, out))
            mdls.append(nn.LeakyReLU())
        
        self.common = nn.Sequential(*mdls)
        feature_out, network_out = sizes[-2], sizes[-1]
        
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(feature_out, network_out),
                nn.ReLU()
            )
            for _ in range(16) # because we predict 16 bits
        ])

    def forward(self, x):
        common = self.common(x)
        return [head(common) for head in self.heads]

In [ ]:
len(dataset12)
#dataset = dataset12

In [ ]:
#datas = SmallestNNDatasetBitListEncoding(SmallestNNDatasetBitEncodedMinusOne2One(dataset))
#datas[0]

In [ ]:
class BinaryReprToBitsExperiment(Experiment):
    def __init__(
        self,
        model = SmallestNNBits(16, 768, 2).to(device),
        train_data = DataLoader(
            SmallestNNDatasetBitListEncoding(SmallestNNDatasetBitEncodedMinusOne2One(dataset12)),
            batch_size=1,
            shuffle=True),
        test_data = DataLoader(
            SmallestNNDatasetBitListEncoding(SmallestNNDatasetBitEncodedMinusOne2One(dataset12)),
            batch_size=1,
            shuffle=True),
        optimizer_class = torch.optim.Adam,
        criterion = torch.nn.MSELoss(),
        device = device
    ):
        super().__init__(
            model, train_data, test_data, optimizer_class, criterion, device
        )
        
        heads_parameters = []
        for name, param in self.model.named_parameters():
            if "head" in name:
                heads_parameters.append(param)
        heads_parameters = nn.ParameterList(heads_parameters)
            
        self.opt_heads = optimizer_class(heads_parameters, lr=1e-4)
        self.sched_heads = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.opt_heads, mode='min', factor=0.95, patience=10,
            verbose=False, threshold=0.01, min_lr=1e-7, eps=1e-08
        )
    
    def train_chunk(self, isTrainCommon: bool = True, headIndex: int = 0):
       
        predictor_nth_bit = nn.Sequential(self.model.common, self.model.heads[headIndex])
        trainable_params = None
        
        if not isTrainCommon:
            trainable_params = nn.Sequential(self.model.heads[headIndex])
        
        return BinaryReprToBitExperiment(
            model = predictor_nth_bit,
            device = self.device,
            epochs=1500,
            init_weigths = isTrainCommon, # if we train common then we need all layers initialized
            trainable_parameters = trainable_params
        )
    
    def train(self, headOrder = []):
        if not headOrder:
            headOrder = list(range(16))
        
        self.bitIdx2Trainer = {}

        self.bitIdx2Trainer[headOrder[0]] = self.train_chunk(
            isTrainCommon=True,
            headIndex=headOrder[0])
        
        for headIdx in range(1, 16):
            self.bitIdx2Trainer[headOrder[headIdx]] = self.train_chunk(
                isTrainCommon=False,
                headIndex=headOrder[headIdx])
        
        for headIdx in headOrder:
            self.bitIdx2Trainer[headIdx].train()
            
        print("After training:")
        for headIdx in headOrder:
            print(f"Head {headIdx} evaluate: -> {self.bitIdx2Trainer[headIdx].evaluate()}")

    def evaluate(self):
        correct_cnt = 0 
        correct_bits = 0
        
        #import pdb; pdb.set_trace()
        for (input, label) in self.test_data:
            label = label.to(self.device).float()
            prediction = self.model(input.to(self.device))[0]
            
            all_bits_correct = True
            for (predicted_bit, target_bit) in zip(label, prediction):
                predicted_bit = predicted_bit.argmax().float()
                target_bit = target_bit.argmax().float()
                
                correct_bit = (predicted_bit == target_bit)
                all_bits_correct = all_bits_correct and correct_bit
                correct_bits += correct_bit

            correct_cnt += float(all_bits_correct)

        return (correct_cnt / len(self.test_data)), correct_bits

In [ ]:
"""
trn2 = BinaryReprToBitsExperiment()
trn2.train_chunk(isTrainCommon=True, headIndex=8).train()
trn2.train_chunk(isTrainCommon=False, headIndex=8).evaluate()
trn2.train_chunk(isTrainCommon=False, headIndex=15).evaluate()
trn2.train_chunk(isTrainCommon=False, headIndex=15).train()
trn2.train_chunk(isTrainCommon=False, headIndex=15).evaluate()
trn2.train_chunk(isTrainCommon=False, headIndex=8).evaluate()
"""

In [ ]:
trn2 = BinaryReprToBitsExperiment()
trn2.train(list(range(16)))

In [ ]:
trn = BinaryReprToBitsExperiment()
trn.train([8, 15, 14, 0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13])

In [ ]:
#trn = BinaryReprToBitsExperiment()
#trn.train([8, 15, 0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14])

In [ ]:
#trn.evaluate()

In [ ]:
class BinaryReprToBitsOneHotExperiment(Experiment):
    def __init__(
        self,
        model = SmallestNNBitsOneHot(16, 512, 16).to(device),
        train_data = DataLoader(
            SmallestNNDatasetBitEncodeNormalized2BitOneHot(dataset),
            batch_size=1,
            shuffle=True),
        test_data = DataLoader(
            SmallestNNDatasetBitEncodeNormalized2BitOneHot(dataset),
            batch_size=1,
            shuffle=True),
        optimizer_class = torch.optim.Adam,
        criterion = torch.nn.MSELoss(),
        device = device
    ):
        super().__init__(
            model, train_data, test_data, optimizer_class, criterion, device
        )    

    def train(self, headOrder = []):
        losses = []
        moment = []
        acc = 0

        for e in tqdm(range(self.epochs)):
            cum_loss = 0
            for batch in self.train_data:
                input, label = batch
                input = input.to(self.device)
                label = label.to(self.device)
                
                #import pdb; pdb.set_trace()
                
                predct = self.model(input)
                loss = self.criterion(label, predct)
                self.opt.zero_grad()
                loss.backward()
                self.opt.step()
                cum_loss += loss.item()

            curr_acc, curr_bit_acc = self.evaluate()
            if acc < curr_acc:
                acc = curr_acc
                print("\rEpoch: %5d:" % (e), curr_acc, curr_bit_acc, end='')
            if acc >= 0.9999:
                break
            self.sched.step(curr_acc)

            moment.append(e)
            losses.append(cum_loss / len(self.train_data))
        self.display_learning(moment, losses)

    def evaluate(self):
        total_correct_cnt = 0 
        total_correct_bits = 0
        
        #import pdb; pdb.set_trace()
        for (input, label) in self.test_data:
            label = label.to(self.device).float()
            prediction = self.model(input.to(self.device))
            #import pdb; pdb.set_trace()
            
            label_bits = label.argmax(2)
            predicted_bits = prediction.argmax(2)
            
            correct_bits = (predicted_bits == label_bits).float().sum().item()
            
            total_correct_bits += correct_bits
            total_correct_cnt += float(correct_bits == 16)

        return (total_correct_cnt / len(self.test_data)), total_correct_bits / len(self.test_data)

In [ ]:
trn = BinaryReprToBitsOneHotExperiment()
trn.train()